**Lecturer:** Sipos, Róbert<br>
**E-mail:** istvan.robert.sipos@citi.com<br>
**Web:** http://www.hit.bme.hu/~siposr/

![Robi S](img\SIPOS_Istvan_Robert.jpg)

# Trade compression

Python for Finance, CEU, 29th March, 2019

## Overview

- [Introduction to the concept of trade compression](#Introduction)
- [Problem as linear programming](#Problem)
- [Solving LP](#LP)
- [Summary](#Summary)

## <a name="Introduction">Introduction</a>

*"The size of the credit-default-swap (CDS) market, which stood at a monumental \$57 trillion dollars in June 2008, has grabbed attention. That number has since shrunk dramatically. According to the Bank for International Settlements, the value of outstanding CDS contracts had fallen to \$42 trillion in December. The apparent collapse is largely down to something called trade compression."* (The Economist)

Trade compression is used to reduce the number of contracts that banks have on their books, while keeping the same economic exposure (present value, risks). 

- Unilateral basis: cancelling offsetting contracts in their own portfolio;
- Bilateral basis: firms cancel offsetting involving two parties;
- Multilateral basis: firms cancel offsetting contracts involving multiple parties.

![Multilateral](files/img/multilateral.png)


<font color='red'>**What is the economic motivation for performing compressions?**</font>
- Reducing the number of outstanding contracts (number of trades, total notional).
- Reduction of capital needed to cover trading book risk.
- Mitigating credit/counterparty risk.
- Easier to manage trading book and hedge with a smaller number of positions.
- The issue is even more acute with central clearing, as one position turns into two.

There are firms offering algorithms for compressing trades: TriOptima and CLS are the main market providers.

Compression effectiveness in greatly facilitated by the level of standardization of the underlying product. For instance, if maturities, coupons are standardized, as it is the case for CDS, then finding offsetting positions is more straightforward.

### Unilateral case

Compression is an optimization problem, where the task is to find a portfolio that is equivalent to the original one, while minimalizing a measure (e.g. number of trades). Equivalence could be defined with a set of boundary conditions:
- Present Value
- Credit Risk
Can be prescribed to remain constant.

In our case, the goal is to reduce the number of securities in a portfolio to minimize the computational cost for our simulations, while keeping these additive risk profiles constant:
- Current Market Price (MTM) of the securities;
- First and higher order derivatives of the market price.
The new portfolio will be a linear combination of the original.

Unfortunately, solving this is a NP-complete problem. Instead, we will investigate a related question that approximates this: **minimalizing the total absolute notional**.

**Definitions:**

- We have $𝑁$ trades in a portfolio.
- Let’s denote the vector of notionals corresponding to these by $ \bf{c} = \left [ c_1, ..., c_N \right ] $.
- We have $𝑀$ risk factors for each trade, e.g. market price (MtM) and it’s derivatives for different tenors.
- In matrix $\bf{A}$ for the $i$th trade risk factor $j$ is denoted by $ A_{i,j} $.
- The sum of the risk factors over the whole portfolio is: $ \bf{\Sigma} = \left [ \Sigma_1, ..., \Sigma_M \right ] $, where $ \Sigma_j = \sum_{i=1}^{N} {A_{i,j}}$.

In [1]:
import pandas as pd
import numpy as np

trades = pd.DataFrame.from_csv('trades.csv')

N = trades.shape[0]
M = trades.shape[1] - 1
c = trades.iloc[:, 0].values
A = trades.iloc[:, 1:].values
buckets = np.sum(A, axis=0)

trades

,Notional,MtM,1m,2m,3m,4m,5m,6m,7m,8m,9m,10m,11m,1y,2y,3y,4y,5y,10y
ID,,,,,,,,,,,,,,,,,,,
1,8939000,214000,-27.445889,-27.395356,0.008122,0.013443,0.022489,193.741103,-164.549612,-78.360250,0.067178,0.075942,0.104629,75.211716,-3.657588,-3.981014,17.706450,-24370.40541,-44315.8051
2,20303000,194000,0.002344,0.008667,-230.708349,-20.587736,0.055208,136.757492,0.096144,0.121115,-593.643977,-104.137295,0.231681,-22100.053600,-15752.164190,0.000000,0.000000,0.00000,0.0000
3,49091000,125000,0.005444,0.021997,-1385.779309,-25247.804960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000
4,12121000,-554000,-0.001229,-0.005005,84.196562,92.784291,-0.027784,-260.792042,-0.063009,-0.077188,207.114581,195.636718,-0.121554,-292.496037,25183.826820,14993.233190,0.000000,0.00000,0.0000
5,11364000,-25000,-0.001336,-0.004359,75.077018,88.405846,-0.030072,-279.241606,-0.056097,-0.074357,174.851585,177.139560,-0.124359,-253.655132,-1.779128,8625.102384,45246.077190,0.00000,0.0000
6,73030000,668000,-49.842202,-459.514072,0.064966,0.113339,0.172443,1591.641604,-245.792940,-1846.942530,0.585084,0.635786,0.853443,885.925116,-168690.638400,-49496.534580,0.000000,0.00000,0.0000
7,51288000,-120000,-0.005644,13179.291540,1168.727946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000
8,51288000,-113000,-0.006164,11679.953070,4230.864395,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000
9,51288000,-116000,-0.005297,10823.608640,4223.470057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000


We seek a vector of portfolio weights $ \bf{w} = \begin{bmatrix} w_1 \\ ... \\ w_N \end{bmatrix} $ with the boundary condition:
$$ \forall i: 0 \leq w_i \leq w_{max} $$
$$ \bf{\Sigma} + \bf{L} \leq \bf{A}^T \bf{w} \leq \bf{\Sigma} + \bf{U} , $$
where $\bf{L}$ and $\bf{U}$ are tolerance levels defined by the user.

**Task: play around with vector $\bf{w}$ and investigate the results with a unit vector, a random weight vector, etc.**

In [ ]:
w_max = 2
L = np.repeat([-300000, -300, -1000], [1, 6, M-7])
U = np.repeat([100000, 100, 300], [1, 6, M-7])

def print_summary(w, c=c, A=A, buckets=buckets, L=L, U=U):  
    print("Original number of trades: %d" % N)
    print("Compressed number of trades: %d" % np.count_nonzero(w))
    print("Compression (#trades): %1.2f%%" % (100.0*np.count_nonzero(w)/N))
    print("Original notional: $%.2f" % np.sum(c))
    print("Compressed notional: $%.2f" % np.dot(c, w))
    print("Compression (notional): %1.2f%%" % (100.0*np.dot(c, w)/np.sum(c)))

    buckets_new = np.dot(A.T, w)
    difference = buckets_new - buckets
    eps = np.finfo(np.float32).eps
    feasible = ((difference >= L-eps) & (difference <= U+eps))
    return pd.DataFrame([buckets, buckets_new, difference, feasible],
                        columns=trades.columns[1:],
                        index=['Sum', 'Sum (new)', 'Difference', 'Feasible'])

# TODO <<<
w = ...
# TODO >>>
print_summary(w)

## <a name="LP">Problem as linear programming</a>

Minimalizing the total notional under boundary conditions:

$$ \bf{w}: \mathop {\arg \min}\limits_{\bf{w}} {\bf{c}^T \bf{w}} $$
such that
$$ \forall i: 0 \leq w_i \leq w_{max} $$
$$ \bf{\Sigma} + \bf{L} \leq \bf{A}^T \bf{w} \leq \bf{\Sigma} + \bf{U} $$
or in a canonical form
$$ \begin{bmatrix} \bf{A}^T \\ - \bf{A}^T \end{bmatrix} \bf{w} \leq \begin{bmatrix} \bf{\Sigma} + \bf{U} \\ - \left(\bf{\Sigma} + \bf{L}\right) \end{bmatrix} $$

<font color='red'>Such task is called LP optimization problem.</font>

**Properties:**
- Initial solution: $\bf{𝐰}=\bf{𝟏}$ (not optimal, but feasible)
- Solution is not unique
- Solution tends to be sparse, i.e. the number of non-zero elements are small

### How to solve LP problems – Simplex algorithm

**Graphical method:**
1. Construct the smallest convex hull based on the constraints
2. Walk on the edges towards increasing/decreasing objective function (greedy algorithm, too many vertices for exhaustive search)

2-D | N-D
- | -
![2-D](files/img/simplex_2d.png) | ![N-D](files/img/simplex_nd.png)

**Properties:**
- Global optimum is guaranteed
- Usually efficient, worst-case exponential
- Listed as one of the top 10 algorithms in the 20th century (by IEEE)

#### Scipy implementation

```python
scipy.optimize.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='simplex', callback=None, options={'disp': False, 'bland': False, 'tol': 1e-12, 'maxiter': 1000})
```

Input:
- c = $ \bf{c} $
- A_ub = $ \begin{bmatrix} \bf{A}^T \\ - \bf{A}^T \end{bmatrix} $
- b_ub = $ \begin{bmatrix} \bf{\Sigma} + \bf{U} \\ - \left(\bf{\Sigma} + \bf{L}\right) \end{bmatrix} $
- bounds = $ \forall i: \left ( 0, w_{max} \right ) $

Returns: a ```scipy.optimize.OptimizeResult```, where
- x = $ \bf{w} $
- fun = $ {\bf{c}^T \bf{w}} $
- success: True if the algorithm succeeded in finding an optimal solution

More details: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/optimize.linprog-simplex.html)

**Task: please implement the LP optimization below with SciPy.**

In [3]:
from scipy.optimize import linprog

def opt_lp(c=c, A=A, buckets=buckets, L=L, U=U, w_max=w_max, N=N):
    # TODO <<<
    return np.ones(N)
    # TODO >>>

w = opt_lp()
print_summary(w)

Original number of trades: 260
Compressed number of trades: 260
Compression (#trades): 100.00%
Original notional: $11012684000.00
Compressed notional: $11012684000.00
Compression (notional): 100.00%


,MtM,1m,2m,3m,4m,5m,6m,7m,8m,9m,10m,11m,1y,2y,3y,4y,5y,10y
Sum,-1.793e+06,12895.5,-15106.1,-27949.3,-112784,-116825,-56854.2,-291197,-63543.5,-167332,79957.9,207173,185405,798160,-2772.79,-233927,-557715,-36244.6
Sum (new),-1.793e+06,12895.5,-15106.1,-27949.3,-112784,-116825,-56854.2,-291197,-63543.5,-167332,79957.9,207173,185405,798160,-2772.79,-233927,-557715,-36244.6
Difference,0,0,1.63709e-11,5.82077e-11,2.91038e-11,2.91038e-11,-7.27596e-12,5.82077e-11,5.82077e-11,5.82077e-11,-4.36557e-11,-2.91038e-11,5.82077e-11,4.65661e-10,1.81899e-11,-1.16415e-10,2.32831e-10,0
Feasible,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


### Other LP algorithms

**Interior point methods:**
- Can visit any point in the feasible region, not only vertices.
- Similar efficiency as the simplex algorithm, depends on the actual problem.

**Approximate algorithms:**  solution is 𝑂(1+𝜀) optimal [1]
Matrix A is 𝑛 x 𝑚, and has N non-zero entries
Best sequential takes 𝑂(𝑁+(𝑙𝑜𝑔𝑁)(𝑛+𝑚) 𝜀^(−2) ) time
Best parallel takes 𝑂((𝑙𝑜𝑔𝑁)^2 𝜀^(−3) ) time

## <a name="Summary">Summary</a>

- Trade compression is therefore an important means of reducing gross notional amounts
- For achieving regulatory capital savings; and reducing operational and counterparty risk exposures. 
- When used on a multilateral basis, for example, with cleared OTC derivatives trades, it also cuts back on double counting of risk. 
- The unilateral trade compression saved a lot of computational resources when calculating CVA.
- Therefore the importance of trade compression should not be underestimated.

### Career Opportunities

* https://jobs.citi.com/job/budapest/summer-intern-2019-markets-quantitative-analysis/287/9592144